In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [2]:
# 加载和准备数据
data = pd.read_csv('earthquake.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M:%S').dt.time
data['Timestamp'] = data.apply(lambda row: pd.Timestamp(f"{row['Date']} {row['Time']}"), axis=1)
data['Timestamp'] = data['Timestamp'].view('int64') // 10**9 #Unix时间戳
features = data[['Timestamp', 'Longitude', 'Latitude']]
targets = data[['Depth', 'Magnitude']]

In [3]:
X_np=features.values
y_np=targets.values

In [4]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
scaler_x.fit(X_np)
scaler_y.fit(y_np)

# 使用相同的参数对训练数据和测试数据进行归一化
X_normalized = scaler_x.transform(X_np)
y_normalized = scaler_y.transform(y_np)

In [5]:
def create_sliding_windows(X, Y, window_size):
    X_windows, Y_windows = [], []
    for i in range(len(X) - window_size):
        # 从X中提取窗口
        X_window = X[i:i+window_size]
        # 从Y中提取紧接着窗口的下一个值作为标签
        Y_window = Y[i+window_size]
        X_windows.append(X_window)
        Y_windows.append(Y_window)
    return np.array(X_windows), np.array(Y_windows)

window_size = 10
X_windows, Y_windows = create_sliding_windows(X_normalized, y_normalized, window_size)

In [6]:
X = torch.tensor(X_windows, dtype=torch.float32)
Y = torch.tensor(Y_windows, dtype=torch.float32)

In [7]:
# 确定测试集的大小
test_size = 0.2

# 计算测试集应有的数据点数量
num_data_points = len(X)
num_test_points = int(num_data_points * test_size)

# 计算测试集和训练集的分割点
split_point = num_data_points - num_test_points

# 按顺序划分数据为训练集和测试集
X_train = X[:split_point]
X_test = X[split_point:]
y_train = Y[:split_point]
y_test = Y[split_point:]

In [8]:
X_train.shape

torch.Size([18722, 10, 3])

In [9]:
# 创建DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

In [15]:
batch_size=64
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

In [18]:
class ComplexLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_prob):
        super(ComplexLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # 定义LSTM层
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        
        # 添加Dropout层
        self.dropout = nn.Dropout(dropout_prob)
        
        # 定义额外的全连接层
        self.fc1 = nn.Linear(hidden_dim, hidden_dim * 2)
        self.bn1 = nn.BatchNorm1d(hidden_dim * 2)  # 批量归一化层
        self.fc2 = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, x):
        # 初始化隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        # 前向传播LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # 只需要LSTM最后一层的输出
        out = out[:, -1, :]
        out = self.dropout(out)
        
        # 通过全连接层
        out = self.fc1(out)
        out = self.bn1(out)
        out = torch.relu(out)
        out = self.fc2(out)
        return out

# 初始化模型
input_dim = 3 # 维度、经度、时间
hidden_dim = 64
num_layers = 2
output_dim = 2 # 震深和震级
dropout_prob = 0.5

model = ComplexLSTMModel(input_dim, hidden_dim, output_dim, num_layers, dropout_prob)

# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 将模型移到GPU
model = model.to(device)

In [19]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 设置训练轮数
num_epochs = 200

# 用于保存最佳模型的逻辑
best_loss = np.inf
best_model_path = 'best_model.pth'

In [20]:
# 训练模型
for epoch in range(num_epochs):
    model.train() # 确保模型处于训练模式
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device) # 移动数据到GPU
        optimizer.zero_grad() # 清除过往梯度
        
        outputs = model(inputs) # 前向传播
        
        loss = criterion(outputs, targets) # 计算损失
        loss.backward() # 后向传播，计算梯度
        optimizer.step() # 更新权重
    
    # 打印训练进度
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    # 保存最佳模型
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), best_model_path)

Epoch [10/200], Loss: 0.2044
Epoch [20/200], Loss: 0.1334
Epoch [30/200], Loss: 0.1531
Epoch [40/200], Loss: 0.1134
Epoch [50/200], Loss: 0.1439
Epoch [60/200], Loss: 0.1666
Epoch [70/200], Loss: 0.1461
Epoch [80/200], Loss: 0.0978
Epoch [90/200], Loss: 0.1424
Epoch [100/200], Loss: 0.1326
Epoch [110/200], Loss: 0.1191
Epoch [120/200], Loss: 0.1410
Epoch [130/200], Loss: 0.1639
Epoch [140/200], Loss: 0.1486
Epoch [150/200], Loss: 0.1290
Epoch [160/200], Loss: 0.1413
Epoch [170/200], Loss: 0.1180
Epoch [180/200], Loss: 0.1052
Epoch [190/200], Loss: 0.1470
Epoch [200/200], Loss: 0.1385


In [41]:
print(f'BestLoss: {best_loss:.4f}')

BestLoss: 0.0915


In [21]:
def evaluate_model(model, test_loader):
    model.eval() # 将模型设置为评估模式
    predictions = []
    actuals = []
    with torch.no_grad(): # 不计算梯度
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device) # 移动数据到GPU
            outputs = model(inputs)
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.cpu().numpy())
    
    predictions = np.vstack(predictions)
    actuals = np.vstack(actuals)
    
    # 反标准化以比较实际值
    predictions = scaler_y.inverse_transform(predictions)
    actuals = scaler_y.inverse_transform(actuals)
    
    # 计算均方根误差
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    
    # 计算平均绝对误差
    mae = mean_absolute_error(actuals, predictions)

    return rmse, mae

In [30]:
# 加载最佳模型
model.load_state_dict(torch.load(best_model_path))
model.to(device)  # 确保模型在正确的设备上

# 评估模型
rmse, mae = evaluate_model(model, test_loader)

print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')

Root Mean Squared Error (RMSE): 11.746
Mean Absolute Error (MAE): 8.647


In [40]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm

# 定义Haversine距离的计算函数
def haversine_distance(point1, point2):
    return geodesic(point1, point2).kilometers

# 将经纬度数据转换为DBSCAN需要的球面距离矩阵
def create_distance_matrix(coordinates):
    num_points = len(coordinates)
    distance_matrix = np.zeros((num_points, num_points))
    
    # 创建一个tqdm进度条，total参数设置为距离矩阵的元素总数
    # 由于距离矩阵是对称的，我们只需要计算上三角部分（不包括对角线）
    num_elements = (num_points * (num_points - 1)) // 2
    pbar = tqdm(total=num_elements)
    
    for i, point1 in enumerate(coordinates):
        for j, point2 in enumerate(coordinates):
            if i < j:  # 只计算上三角部分
                distance_matrix[i][j] = haversine_distance(point1, point2)
                distance_matrix[j][i] = distance_matrix[i][j]  # 对称复制到下三角部分
                pbar.update(1)  # 更新进度条
        pbar.refresh()  # 刷新进度条显示
    
    pbar.close()  # 完成时关闭进度条
    return distance_matrix

# 使用DBSCAN进行聚类
def dbscan_clustering(coordinates, eps, min_samples):
    distance_matrix = create_distance_matrix(coordinates)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
    dbscan.fit(distance_matrix)
    return dbscan.labels_

# 加载CSV数据
df = data

# 准备经纬度数据
coordinates = df[['Latitude', 'Longitude']].values

# 进行聚类，这里 eps 和 min_samples 需要根据数据调整
eps_in_kilometers = 1000  # 聚类半径100公里
min_samples = 2000          # 一个聚类中最小的样本数量
labels = dbscan_clustering(coordinates, eps_in_kilometers, min_samples)

# 将聚类标签添加到原始数据中
df['Cluster'] = labels

print(df.head())

100%|███████████████████████████████████████████████████████████████| 274049166/274049166 [12:40:32<00:00, 6005.56it/s]


        Date      Time  Latitude  Longitude        Type  Depth  Magnitude  \
0 1965-01-02  13:44:18    19.246    145.616  Earthquake  131.6        6.0   
1 1965-01-04  11:29:49     1.863    127.352  Earthquake   80.0        5.8   
2 1965-01-05  18:05:58   -20.579   -173.972  Earthquake   20.0        6.2   
3 1965-01-08  18:49:43   -59.076    -23.557  Earthquake   15.0        5.8   
4 1965-01-09  13:32:50    11.938    126.427  Earthquake   15.0        5.8   

   Timestamp  Cluster  
0 -157630542       -1  
1 -157465811        1  
2 -157355642        0  
3 -157093817       -1  
4 -157026430       -1  


In [43]:
df.to_excel('process_earthquake.xlsx')